# SageMaker Custom Container

## SageMaker Container Setup

#### Export the AWS Credentials 

In [ ]:
!export AWS_ACCESS_KEY_ID="<aws_access_key>"
!export AWS_SECRET_ACCESS_KEY="<aws_secret_key>"

#### Change the current directory to the directory containing the Dockerfile 

In [13]:
cd ~/SageMaker/custom-container-folder

/home/ec2-user/SageMaker/docker_test_folder


#### Login to the ECR Docker registry and Build the Custom Container

In [ ]:
! aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 683313688378.dkr.ecr.us-east-1.amazonaws.com
! docker build -t test-custom-sagemaker -f Dockerfile .

#### Login to our aws account, Tag the Docker Image, Push to the Repository

In [15]:
# Login to our aws account (replace the account-id)
! aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin <accountid>.dkr.ecr.us-east-1.amazonaws.com
# Tag the Docker Image (replace account-id and repository-name)
! docker tag <repository-name>:latest <accountid>.dkr.ecr.us-east-1.amazonaws.com/<repository-name>:latest
# Push to the Repository (replace account-id and repository-name)
! docker push <accountid>.dkr.ecr.us-east-1.amazonaws.com/<repository-name>:latest

/bin/sh: accountid: No such file or directory
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='UTF-8'>
BrokenPipeError: [Errno 32] Broken pipe
/bin/sh: repository-name: No such file or directory
/bin/sh: accountid: No such file or directory


## Test the Custom Container

In [17]:
import boto3
import sagemaker
from sagemaker import get_execution_role

region = boto3.session.Session().region_name

role = get_execution_role()

In [18]:
from sagemaker.processing import ScriptProcessor

custom_image_uri = "<docker-URI-from-ECR>"

script_processor = ScriptProcessor(command=['python3'],
                image_uri=custom_image_uri,
                role=role,
                instance_count=1,
                instance_type='ml.m5.large')

In [ ]:
script_processor.run(code='custom_sm_container_test.py')
script_processor_job_description = script_processor.jobs[-1].describe()
print(script_processor_job_description)